In [1]:
import pandas as pd
import math

#pull reviews
reviews = pd.read_csv("reviews.csv")

#take train data...
num_test = math.floor(len(reviews) * .2)
num_train = len(reviews) - num_test

print(len(reviews))
print(num_test)
print(num_train)

2511
502
2009


In [2]:
test = reviews.iloc[:num_test]
train = reviews.iloc[num_test:]

print(len(test))
print(len(train))

502
2009


In [3]:
test_labels = test['rating'].to_frame()
test_text = test['text'].to_frame()

train_labels = train['rating'].to_frame()
train_text = train['text'].to_frame()

In [4]:
import nltk
from nltk.tokenize import WordPunctTokenizer

tk = WordPunctTokenizer()
NEG = 'NOT_'
negs = ['not', 'none', 'no', 'impossible', 'n\'t']
punct = [',', '.', '?', '!', ';', '-']

#preps data
def prep(text):
    data_list = []
    for i, data in text.iterrows():
        words = []
        review = data['text']
        listed = review.split()
        neggy = False
        for word in listed:
            w = word.lower()
            if neggy:
                if word in punct:
                    words.append(w)
                    neggy = False
                else:
                    words.append(NEG + w)
            else:
                words.append(w)
                if w in negs:
                    if neggy:
                        neggy = False
                    else:
                        neggy = True
        data_list.append(' '.join(words))
    return data_list
 
train_list = prep(train_text)
tokens = tk.tokenize(' '.join(train_list))
freq = nltk.FreqDist(tokens)
vocab = {'<unk>':0}
i = 1
for item in freq.keys():
    if item not in vocab:
        if freq[item] > 2:
            vocab[item] = i
            i += 1
print(len(vocab))

11768


In [5]:
import numpy as np

def make_matrix(text_list):
    n = len(text_list)
    X = np.zeros((n, len(vocab)))
    for i in range(n):
        review = tk.tokenize(text_list[i])
        for word in review:
            if word in vocab:
                X[i][vocab[word]] += 1
            else:
                X[i][0] += 1
    return X

train_matrix = make_matrix(train_list)

print("train feature matrix shape: ({}, {})".format(len(train_matrix), len(train_matrix[0])))
print("train label array shape: ({},)".format(len(train_labels)))

train feature matrix shape: (2009, 11768)
train label array shape: (2009,)


In [6]:
#clean test data

test_list = prep(test_text)
test_matrix = make_matrix(test_list)

print("test feature matrix shape: ({}, {})".format(len(test_matrix), len(test_matrix[0])))
print("test label array shape: ({},)".format(len(test_labels)))

test feature matrix shape: (502, 11768)
test label array shape: (502,)


In [7]:
def clean_labels(y):
    for i, val in enumerate(y):
        if val > 3:
            y[i] = 1
        else:
            y[i] = -1
    return y

X_train = train_matrix
y_train = clean_labels(np.ravel(train_labels.to_numpy()))

X_test = test_matrix
y_test = clean_labels(np.ravel(test_labels.to_numpy()))

In [8]:
# OLS

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

lin_regressor=LinearRegression()

mse=cross_val_score(lin_regressor,X_train,y_train,scoring='neg_mean_squared_error',cv=cv)
mean_mse=np.mean(mse)
print(mean_mse)

-3.2562075043509395e+19


In [9]:
X_train = train_matrix
y_train_LS = np.ravel(train_labels.to_numpy())

X_test = test_matrix
y_test_LS = np.ravel(test_labels.to_numpy())

In [10]:
lin_regressor.fit(X_train, y_train)
lin_regressor.predict(X_train)

array([ 1.00026008,  1.00009486, -0.99994023, ...,  1.00014421,
        1.00006762,  1.00010351])

In [11]:
# loss = np.linalg.norm(y_hat_test_OLS - y_test_OLS)


In [12]:
y_hat_LS = lin_regressor.predict(X_train)

def reg_score(y_hat):
    for i, val in enumerate(y_hat):
        if val < 0:
            y_hat[i] = -1
        else:
            y_hat[i] = 1
    return y_hat

y_hat_LS = reg_score(y_hat_LS)
loss = np.linalg.norm(y_hat_LS - y_train)
print(loss)

2.0


In [13]:
y_hat_test = lin_regressor.predict(X_test)
y_hat_test = reg_score(y_hat_test)
loss_test = np.linalg.norm(y_hat_test - y_test)
print(loss_test)

30.659419433511783


In [14]:
#model for number of stars review

test_labels = test['rating'].to_frame()
train_labels = train['rating'].to_frame()

X_train = train_matrix
y_train_LS_nums = np.ravel(train_labels.to_numpy())

X_test = test_matrix
y_test_LS_nums = np.ravel(test_labels.to_numpy())

In [15]:
lin_regressor.fit(X_train, y_train_LS_nums)

lin_regressor.predict(X_train)

array([4.00013108, 4.0000583 , 1.00003934, ..., 5.00007067, 5.00004654,
       5.00005964])

In [16]:
y_hat_LS_nums = lin_regressor.predict(X_train)
loss = np.linalg.norm(y_hat_LS_nums - y_train_LS_nums)
print(loss)

y_hat_testLS_nums = lin_regressor.predict(X_test)
loss_nums = np.linalg.norm(y_hat_testLS_nums - y_test_LS_nums)
print(loss_nums)

0.7071342341556409
3115953151.939873


In [18]:
y_hat_testLS_nums

array([ 8.81191033e+07, -4.29095030e+07, -1.65510144e+08, -6.37107753e+07,
       -2.34130515e+08,  1.68148313e+08, -6.69357742e+07, -1.31499365e+08,
       -3.00946966e+07, -4.48755504e+07, -4.41841789e+08,  5.63314234e+08,
       -5.90015601e+06,  1.73552863e+08,  1.37735323e+08, -1.41683882e+07,
        3.02957867e+08,  8.00071194e+07,  6.25250712e+07,  4.00685101e+07,
        8.05647161e+07, -3.91490366e+08, -1.64211359e+08,  8.20852196e+07,
       -3.38024140e+07,  1.77102380e+07, -5.62557526e+07, -1.63592476e+08,
        1.24730498e+06, -4.21856235e+07, -1.81433287e+08, -1.25236496e+08,
       -1.27240668e+06,  4.70012809e+07,  9.89770402e+07, -9.69699822e+07,
       -1.37200706e+07,  6.27930882e+07,  1.50196159e+08, -2.38523702e+08,
        1.43035691e+08, -4.72908803e+08, -1.01426425e+08,  7.03370424e+07,
        4.49049569e+07,  1.70336895e+08,  2.16654281e+07, -3.48306734e+07,
        1.25163614e+07,  2.11939354e+08, -3.07568236e+07,  2.20114231e+08,
        2.46192129e+08, -

In [19]:
y_test_LS_nums

array([5., 2., 5., 1., 3., 3., 4., 5., 5., 3., 4., 4., 4., 4., 4., 1., 3.,
       5., 3., 5., 5., 4., 5., 3., 4., 4., 5., 5., 5., 2., 3., 5., 5., 4.,
       1., 5., 4., 5., 4., 2., 5., 4., 4., 5., 5., 5., 4., 5., 3., 4., 2.,
       4., 4., 4., 5., 4., 4., 5., 5., 2., 5., 5., 5., 5., 4., 4., 5., 4.,
       3., 1., 2., 3., 3., 3., 4., 5., 5., 4., 4., 5., 3., 4., 3., 5., 4.,
       1., 1., 5., 4., 3., 3., 4., 2., 3., 4., 5., 3., 2., 1., 5., 5., 3.,
       5., 4., 4., 3., 5., 5., 5., 5., 3., 3., 2., 1., 3., 4., 1., 3., 2.,
       5., 2., 2., 3., 4., 4., 5., 5., 3., 5., 4., 5., 3., 5., 5., 4., 2.,
       4., 4., 5., 4., 4., 2., 2., 3., 4., 4., 4., 5., 2., 3., 4., 4., 4.,
       3., 4., 2., 4., 5., 4., 4., 5., 5., 5., 5., 4., 3., 4., 3., 5., 5.,
       5., 5., 4., 3., 2., 5., 3., 5., 5., 5., 5., 5., 5., 4., 4., 1., 4.,
       4., 5., 5., 5., 4., 4., 2., 4., 5., 5., 4., 4., 5., 5., 4., 4., 5.,
       2., 4., 5., 4., 2., 5., 5., 5., 4., 4., 2., 4., 1., 5., 5., 5., 5.,
       5., 5., 1., 5., 5.

In [20]:
y_hat_test

array([ 1., -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1.,  1., -1.,
        1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1., -1.,  1.,
       -1., -1.,  1., -1., -1., -1., -1.,  1.,  1., -1., -1.,  1.,  1.,
       -1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1.,
        1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1.,  1.,
        1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1.,
        1.,  1.,  1., -1.,  1.,  1., -1., -1., -1., -1.,  1., -1., -1.,
        1.,  1.,  1., -1., -1., -1.,  1., -1.,  1.,  1.,  1.,  1., -1.,
        1.,  1.,  1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1., -1.,
       -1., -1., -1., -1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,
       -1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
        1., -1., -1.,  1., -1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1., -1.,  1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,
       -1., -1., -1., -1., -1., -1.,  1., -1.,  1.,  1.,  1., -1

In [22]:
y_test

array([ 1., -1.,  1., -1., -1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,
        1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
        1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,
       -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1., -1., -1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,
        1.,  1., -1.,  1., -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,
        1., -1., -1.,  1.,  1., -1., -1., -1.,  1.,  1., -1.,  1.,  1.,
        1., -1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1.,  1., -1.,
       -1., -1.,  1., -1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
        1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1., -1.,
       -1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1.,  1., -1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,
        1.,  1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1.,  1